<a href="https://colab.research.google.com/github/ChanJianHao/SubMe/blob/master/SubMe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SubMe

Review the **Processing Settings** below, then click `Runtime > Run All` to start the program.

Github URL: [repo1](https://github.com/ChanJianHao/SubMe/blob/master/SubMe.ipynb)



# Project Setup

It may take a few minutes to load the program. Please be patient.

In [ ]:
#@title Project Settings

import os

#@markdown Repo path to clone from.
repo_url = 'https://github.com/BingLingGroup/autosub' #@param {type:"string"}
# repo_name = repo_url.split('/')[-1]

#@markdown Repo branch or tag to clone. Leave blank to clone `master`.
repo_branch = 'alpha' #@param {type:"string"}

#@markdown Root workspace.
root_dir = '/content/subme' #@param {type:"string"}
repo_dir = os.path.join(root_dir, repo_name)

#@markdown Temp directory. If your root workspace is in Google Drive, then this path should be a local folder, such as `/content`.
temp_dir = '/content/subme/work' #@param {type:"string"}

#@markdown Output directory.
output_dir = '/content/subme/work/output' #@param {type:"string"}

#@markdown Attempt to request Google Drive access. Will mount to `/content/drive/My Drive`. 
#@markdown Will save output to your linked Google Drive account inside `subme` folder. 
#@markdown
#@markdown Try auto-mounting by clicking the Files button on the left, then click `Mount Drive`.
request_drive = True #@param {type:"boolean"} 

#@markdown Install `dependencies.txt` from project.
install_deps = True #@param {type:"boolean"}

#@#markdown Attempt to invoke high memory (25.5 GB) in Colab.
high_memory = False #@#param {type:"boolean"} 


In [ ]:
#@title System Setup
#@markdown Attempt to invoke high memory if enabled in settings. Trigger Tensorflow version.

!pip install gputil

# Import packages
import os,sys,humanize,psutil,GPUtil

def mem_report():
  print("CPU RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ))
  
  GPUs = GPUtil.getGPUs()
  for i, gpu in enumerate(GPUs):
    print('GPU {:d} ... Mem Free: {:.0f}MB / {:.0f}MB | Utilization {:3.0f}%'.format(i, gpu.memoryFree, gpu.memoryTotal, gpu.memoryUtil*100))
    
mem_report()

if high_memory and psutil.virtual_memory().available < 13958643712:
  print('Attempting to invoke high memory.')
  print('This notebook will crash intentionally and Colab should display a prompt to offer you high-RAM.')
  print('IF THIS PROMPT DOES NOT SHOW, DISABLE high_memory IN THE SETTINGS!')
  d=[]
  while(1):
    d.append('1')

In [ ]:
#@title Project Setup
#@markdown Setup directories. Clone project from git.

from google.colab import drive

if request_drive:
  drive.mount('/content/drive')
output_dir = ('/content/drive/My Drive/subme')

os.makedirs(root_dir, exist_ok=True)
os.makedirs(temp_dir, exist_ok=True)
os.makedirs(output_dir, exist_ok=True)

if not os.path.isdir(repo_dir):
  repo_cmd = f'--branch {repo_branch}' if repo_branch else ''
  !git clone {repo_url} {repo_cmd} --depth 1 "{repo_dir}"

In [ ]:
#@title Dependency Setup
#@markdown Install dependencies from `requirements.txt`.

if install_deps:
  %cd "{root_dir}/{repo_name}"
  %mkdir "src"
  %cd "src"

  # Strip tensorflow from requirements list because Colab provides this custom-compiled.
  # !pip install -r requirements.txt
  # !apt install ffmpeg python python-pip git -y
  # !pip install git+https://github.com/BingLingGroup/autosub.git@origin
  !apt install ffmpeg python3 curl git -y
  !curl https://bootstrap.pypa.io/get-pip.py -o get-pip.py
  !python3 get-pip.py
  !pip install git+{repo_url}@{repo_branch} ffmpeg-normalize langcodes

# Upload Videos for Processing

In [ ]:
#@#title Upload Videos for Processing
#@markdown Click `Runtime > Run All` to upload images. If you see an error in this section, then run this cell again.

from google.colab import files
import shutil

if request_drive:
  uploaded = []
  file_no = int(input("No. of files to sub: "))
  for i in range(file_no):
    file_name = input("Please enter file name with extension: ")
    file_name = "/content/drive/My\ Drive/subme/" + file_name
    uploaded.append(file_name)
else:
  uploaded = files.upload()
  uploaded = list(uploaded.keys())

print(uploaded)


In [ ]:
print(uploaded)

# Processing

In [ ]:
#@markdown Choose media source language.

src_language = 'en' #@param {type:"string"}
dst_language = 'en' #@param {type:"string"}

for file in uploaded:
  print('User uploaded file "{name}" '.format(
      name=file))
  !autosub -i {file} -S {src_language} -D {dst_language}

# !autosub -i {uploaded} -S {src_language} -D {dst_language}

